# CSCE 5380 Spring 2018 - Data Mining - Lab Assignment 1  - Task 1

Constant Marks

<h1>Index<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-1:-Cross-validation" data-toc-modified-id="Problem-1:-Cross-validation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem 1: Cross-validation</a></span><ul class="toc-item"><li><span><a href="#Question-1." data-toc-modified-id="Question-1.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Question 1.</a></span></li><li><span><a href="#Question-2" data-toc-modified-id="Question-2-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Question 2</a></span></li><li><span><a href="#Question-3." data-toc-modified-id="Question-3.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Question 3.</a></span></li></ul></li><li><span><a href="#Problem-2:-Percentage-Split" data-toc-modified-id="Problem-2:-Percentage-Split-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Problem 2: Percentage Split</a></span><ul class="toc-item"><li><span><a href="#Question-1." data-toc-modified-id="Question-1.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Question 1.</a></span></li><li><span><a href="#Question-2." data-toc-modified-id="Question-2.-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Question 2.</a></span></li><li><span><a href="#Question-3." data-toc-modified-id="Question-3.-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Question 3.</a></span></li></ul></li></ul></div>

## Problem 1: Cross-validation

We will first run the Naïve Bayes classifier with 3 different numbers of folds (5, 10, 15 folds in each run respectively) and then run the Random Forest classifier by selecting the same number of folds. Next we will choose a 10% test set split, fit the classifiers, predict the results of the tes set, and compare the predicted results to the known values.

First we import the data and inspect the first record in the data set.

In [3]:
from scipy.io.arff import loadarff
with open('data/splice.arff', 'r') as f:
    data, meta = loadarff(f)
print(data[0],'\n')
print('There are %d records: ' % (data.size))

(b'ATRINS-DONOR-521', b'C', b'C', b'A', b'G', b'C', b'T', b'G', b'C', b'A', b'T', b'C', b'A', b'C', b'A', b'G', b'G', b'A', b'G', b'G', b'C', b'C', b'A', b'G', b'C', b'G', b'A', b'G', b'C', b'A', b'G', b'G', b'T', b'C', b'T', b'G', b'T', b'T', b'C', b'C', b'A', b'A', b'G', b'G', b'G', b'C', b'C', b'T', b'T', b'C', b'G', b'A', b'G', b'C', b'C', b'A', b'G', b'T', b'C', b'T', b'G', b'EI') 

There are 3190 records: 


Next we slice out the feature set, encode the values, and cast the values to integers.

In [4]:
X = data[meta.names()[1:-1]]
import numpy as np
X = np.asarray(X.tolist())
features = np.unique(X)
print('Features: ',features)

from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
labelencoder_X.fit(features)
for i in range(np.shape(X)[1]):
    X[:,i] = labelencoder_X.transform(X[:, i])
X = X.astype(int)
print('Encoded to: ',np.unique(X))

Features:  [b'A' b'C' b'D' b'G' b'N' b'R' b'S' b'T']
Encoded to:  [0 1 2 3 4 5 6 7]


Now we slice out the dependanet variable, encode the data (which is not required for these models but may be useful later) and count the freqeuncy of each class.

In [5]:
y = data[meta.names()[-1]]
labels = np.unique(y).astype(str)
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
bin_count = np.bincount(y)
label_freq = np.array([labels, bin_count])
print('Class Frequencies: \n',label_freq)


Class Frequencies: 
 [['EI' 'IE' 'N']
 ['767' '768' '1655']]


Now we create the Naive Bayes Classifier and perform k-fold cross validation.


In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

nb_classifier = GaussianNB()
nb_scores_5 = cross_val_score(nb_classifier, X, y, cv=5)
nb_scores_10 = cross_val_score(nb_classifier, X, y, cv=10)
nb_scores_15 = cross_val_score(nb_classifier, X, y, cv=15)

print("Naive Bayes Cross Validation(k=5) Accuracy:  %0.2f (+/- %0.2f)" % (nb_scores_5.mean(), nb_scores_5.std() * 2))
print("Naive Bayes Cross Validation(k=10) Accuracy: %0.2f (+/- %0.2f)" % (nb_scores_10.mean(), nb_scores_10.std() * 2))
print("Naive Bayes Cross Validation(k=10) Accuracy: %0.2f (+/- %0.2f)" % (nb_scores_15.mean(), nb_scores_15.std() * 2))

Naive Bayes Cross Validation(k=5) Accuracy:  0.91 (+/- 0.01)
Naive Bayes Cross Validation(k=10) Accuracy: 0.91 (+/- 0.03)
Naive Bayes Cross Validation(k=10) Accuracy: 0.91 (+/- 0.03)


And now we create the Random Forest Classifier and perform k-fold cross validation.

In [7]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
rf_scores_5 = cross_val_score(rf_classifier, X, y, cv=5)
rf_scores_10 = cross_val_score(rf_classifier, X, y, cv=10)
rf_scores_15 = cross_val_score(rf_classifier, X, y, cv=15)

print("Random Forest Cross Validation(k=5) Accuracy:  %0.2f (+/- %0.2f)" % (rf_scores_5.mean(), rf_scores_5.std() * 2))
print("Random Forest Cross Validation(k=10) Accuracy: %0.2f (+/- %0.2f)" % (rf_scores_10.mean(), rf_scores_10.std() * 2))
print("Random Forest Cross Validation(k=10) Accuracy: %0.2f (+/- %0.2f)" % (rf_scores_15.mean(), rf_scores_15.std() * 2))

Random Forest Cross Validation(k=5) Accuracy:  0.94 (+/- 0.01)
Random Forest Cross Validation(k=10) Accuracy: 0.95 (+/- 0.03)
Random Forest Cross Validation(k=10) Accuracy: 0.95 (+/- 0.04)


### Question 1.

Do the number of folds have any correlation with the number and percentage of correctly classified instances within the same model (For example, 5 folds and 10 folds in NB and RF respectively)? Explain the results.

   **Answer:** Yes by selecting a smaller number of folds the accuracy of the Naive Bayes classifier does increase slightly (but not statically significant).  This is because with a smaller number of folds we have a larger training set to fit the model, and with a probalistic model this will typically increase the accuracy wthout overfitting.  
    
   With the Random Forest Classifier, however, the accuracy does not increase with increasing folds and may be even be degraded, indicating that the Random Forest model is overfitting with increased training set size.

### Question 2

Do the same number of folds when applied to different models have any effect on the number and percentage of correctly classified instances (For example, 5 folds and 10 folds in NB and RF)? Explain the results.

In [24]:
nb_variance = np.array([nb_scores_5.var(), nb_scores_10.var(), nb_scores_15.var()])
print(nb_variance)
rf_variance = np.array([rf_scores_5.var(), rf_scores_10.var(), rf_scores_15.var()])
print(rf_variance)
d=np.abs((nb_variance-rf_variance))
print(d)
mean_d = d.mean()
k = d.size
difsumsq_d = np.sum((d - mean_d)**2)
varsq_d = difsumsq_d/(k/(k-1))

[  3.19746840e-05   2.89503850e-04   3.05751038e-04]
[  3.93652691e-05   1.56835008e-04   3.89675500e-04]
[  7.39058512e-06   1.32668842e-04   8.39244621e-05]


   **Answer:**Yes, clearly the models will behave differently even with the same number of folds because they are based on different classification algorithms. Also as mentioned above, increasing the number of folds does not improve the accuracy of the Naive Bayes model but may increase the accuracy of the Random Forest model.

### Question 3

Select 1 set of results generated for each classifier. For example, if you performed a test by selecting 10 folds, select the results you obtained for 10 folds for both–NB and RF. Considering all classes in the dataset; calculate the accuracy and error rate for the results of NB and RF. Show the formula and explain the steps in calculating the accuracy and error-rate. **Hint**: compute the values of the confusion matrix first.

Now we will select a 10% test split for each model, fit the models to the training set, predict the results of the test set,  and evaluate the accuracy of the predictions. 

In [84]:
# Splitting data into test and training set choosing a 10% sample Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

# Fitting the Naive Bayes classifier
nb_classifier.fit(X_train,y_train)

# Predicting the Test set results with the Naive Bayes classifier
y_pred = nb_classifier.predict(X_test)

# Making the Confusion Matrix and Calculatng Accuracy Naive Bayes classifier
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
nb_accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm)
nb_error = 1 - nb_accuracy 

# Format the output
print('The Naive Bayes Classifier with a 10%% test split has a single test accuracy rate of %.3f,\n an error rate of %.3f, and a k-fold cross validation score of %.3f (+/- %0.2f)\n' % (nb_accuracy, nb_error, nb_scores_10.mean(),nb_scores_10.std()*2))


# Fitting the Random Forest classifier
rf_classifier.fit(X_train,y_train)

# Predicting the Test set results with the Random Forest classifier
y_pred = rf_classifier.predict(X_test)

# Making the Confusion Matrix and Calculatng Accuracy of the Random Forest classifier
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
rf_accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm)
rf_error = 1 - rf_accuracy 

# Format the output
print('The Random Forest Classifier with a 10%% test split has a single test accuracy rate of %.3f,\n an error rate of %.3f, and a k-fold cross validation score of %.3f (+/- %0.2f)' % (rf_accuracy, rf_error, rf_scores_10.mean(),rf_scores_10.std()*2))


The Naive Bayes Classifier with a 10% test split has a single test accuracy rate of 0.928,
 an error rate of 0.072, and a k-fold cross validation score of 0.912 (+/- 0.03)

The Random Forest Classifier with a 10% test split has a single test accuracy rate of 0.953,
 an error rate of 0.047, and a k-fold cross validation score of 0.944 (+/- 0.02)


   **Answer:** From above you can see the Naive Bayes and Random Forest models results with a 10% test split. Where accuracy = (sum correct predictions) / (sum total predictions). The correct predictions are taken from the diagonal of the confusion matrix and the total is the sum of of all the elements in the confusion matrix. The error rate is simply, error =  1 - accuracy.

## Problem 2: Percentage Split

Now we will run the NB anf RF classifiers by with three different percentages of training data: 25%-75%, 50%-50%, and 75%-25%, and determine if the percentage of training data affects the classifier accuracy?

In [85]:
# Splitting data into three different test and training sets 
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size = .25)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size = .5)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, test_size = .75)

In [87]:
# Fitting and evaluating the Naive Bayes classifier with the 25% test set
nb_classifier.fit(X_train_1,y_train_1)
y_pred = nb_classifier.predict(X_test_1)
cm = confusion_matrix(y_test_1, y_pred)
nb_accuracy = np.array([(cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm),0,0])
nb_error = np.array([(1 - nb_accuracy[0]),0,0])


# Fitting and evaluating the Naive Bayes classifier with the 50% test set
nb_classifier.fit(X_train_2,y_train_2)
y_pred = nb_classifier.predict(X_test_2)
cm = confusion_matrix(y_test_2, y_pred)
nb_accuracy[1] = (cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm)
nb_error[1] = 1 - nb_accuracy[1]


# Fitting and evaluating the Naive Bayes classifier with the 75% test set
nb_classifier.fit(X_train_3,y_train_3)
y_pred = nb_classifier.predict(X_test_3)
cm = confusion_matrix(y_test_3, y_pred)
nb_accuracy[2] = (cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm)
nb_error[2] = 1 - nb_accuracy[2]


# Fitting and evaluating the Random Forest classifier with the 25% test set
rf_classifier.fit(X_train_1,y_train_1)
y_pred = rf_classifier.predict(X_test_1)
cm = confusion_matrix(y_test_1, y_pred)
rf_accuracy = np.array([(cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm),0,0])
rf_error = np.array([(1 - rf_accuracy[0]),0,0])

# Fitting and evaluating the  Random Forest classifier with the 50% test set
rf_classifier.fit(X_train_2,y_train_2)
y_pred = rf_classifier.predict(X_test_2)
cm = confusion_matrix(y_test_2, y_pred)
rf_accuracy[1] = (cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm)
rf_error[1] = 1 - rf_accuracy[1]

# Fitting and evaluating the  Random Forest classifier with the 75% test set
rf_classifier.fit(X_train_3,y_train_3)
y_pred = rf_classifier.predict(X_test_3)
cm = confusion_matrix(y_test_3, y_pred)
rf_accuracy[2] = (cm[0,0]+cm[1,1]+cm[2,2])/np.sum(cm)
rf_error[2] = 1 - rf_accuracy[2]


    

print('Naive Bayes Accuracy: \n 25-75\t 50-50\t 75-25 \n',np.around(nb_accuracy,3))
print ('Naive Bayes Error: \n 25-75\t 50-50\t 75-25 \n',np.around(nb_error,3),'\n')
print('Random Forest Accuracy: \n 25-75\t 50-50\t 75-25 \n',np.around(rf_accuracy,3))
print ('Random Forest Error: \n 25-75\t 50-50\t 75-25 \n',np.around(rf_error,3))



Naive Bayes Accuracy: 
 25-75	 50-50	 75-25 
 [ 0.892  0.907  0.894]
Naive Bayes Error: 
 25-75	 50-50	 75-25 
 [ 0.108  0.093  0.106] 

Random Forest Accuracy: 
 25-75	 50-50	 75-25 
 [ 0.941  0.95   0.93 ]
Random Forest Error: 
 25-75	 50-50	 75-25 
 [ 0.059  0.05   0.07 ]


In [89]:
from prettytable import PrettyTable

x = PrettyTable()

x.field_names = ["City name", "Area", "Population", "Annual Rainfall"]

x.add_row(["Adelaide", 1295, 1158259, 600.5])
x.add_row(["Brisbane", 5905, 1857594, 1146.4])
x.add_row(["Darwin", 112, 120900, 1714.7])
x.add_row(["Hobart", 1357, 205556, 619.5])
x.add_row(["Sydney", 2058, 4336374, 1214.8])
x.add_row(["Melbourne", 1566, 3806092, 646.9])
x.add_row(["Perth", 5386, 1554769, 869.4])


ModuleNotFoundError: No module named 'prettytable'

### Question 1.

Does the percentage of training data affect the classifier accuracy? How and why?
    
   **Answer** Yes, the percentage of the training set does effect the classifier accuracy for the same reasons discussed above for k-fold cross validation.  The Naive Bayes classifier improves with a larger training set due to the probalistic combination of the data.  The Random Forest classifier does not improve because with increasing training set size.  This indicates that the optimum tree size is created with only a small portion of the data set and any additional data will cause overfitting.

### Question 2.

If the same percentage of testing data is used for both classifiers (e.g. 40% for both NB and RF, does the classification accuracy vary from one classifier to another? Why?
    
   **Answer** Yes, if we observe the 50-50 split of the data set, we see that the Naive Bayes and Random Forest classifiers had an 82% and 90% accurary rate respectivey. It is clear that the Naive Bayes classifier needs a larger training set to predict with comparale accuray to the Random Forest classifier.

### Question 3.

Select any one set of results generated for each classifier. For example, if you performed a test by selecting 40% training data, select the results you obtained for 40% for both – NB and RF. Considering all classes in the dataset, calculate the accuracy and error rate for the results of NB and RF. Show the formula and explain the steps in calculating the accuracy and error-rate.
    
   **Answer** The accuracy and error rate ar shown above for all the splits.  The formula for the accuracy and error rate are discussed in Part 1.  If we consider the 50-50 split, the Naive Bayes accurary rate is 82% and the error rat is 18%, and the Random Forest accuracy rate is 90% and the error rate is 10%.